# Check raw data structure
Take a look at the first 5 lines of a typical dataset in dbs/raw_data_se_2019/.

In [1]:
%load_ext autoreload
%autoreload 2
%cd D:\mad4abm

D:\mad4abm


In [2]:
import pandas as pd
from datetime import datetime
from tzwhere import tzwhere
import os
import skmob
from skmob.preprocessing import filtering
from skmob.preprocessing import detection

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
chunk_container = pd.read_csv("dbs/raw_data_se_2019/06/01/locations-00-part0000.csv.gz", sep='\t', chunksize=5000, compression='gzip')
for chunk in chunk_container:
    print(chunk.columns)
    break

Index(['timestamp', 'device_aid', 'device_aid_type', 'latitude', 'longitude',
       'horizontal_accuracy', 'altitude', 'altitude_accuracy',
       'location_method', 'ip', 'user_agent', 'OS', 'OS_version',
       'manufacturer', 'model', 'carrier'],
      dtype='object')


In [5]:
tdf = skmob.TrajDataFrame(chunk.iloc[[0]], user_id='device_aid', datetime='timestamp',
                              latitude='latitude', longitude='longitude')
stdf = detection.stops(filtering.filter(tdf, max_speed_kmh=500.),
                           stop_radius_factor=0.5, no_data_for_minutes=60,
                           minutes_for_a_stop=10.0, min_speed_kmh=3,
                           spatial_radius_km=0.1, leaving_time=True)
stdf

,datetime,uid,device_aid_type,lat,lng,horizontal_accuracy,altitude,altitude_accuracy,location_method,ip,user_agent,OS,OS_version,manufacturer,model,carrier,leaving_datetime


In [8]:
from timezonefinder import TimezoneFinder
def convert_to_location_tz(row, _tf=TimezoneFinder()):
    # if lat/lon aren't specified, we just want the existing name (e.g. UTC)
    if (row.latitude == 0) & (row.longitude == 0):
        return (row.TimeUTC.tz_localize('UTC').tzname(), row.TimeUTC)
    # otherwise, try to find tz name
    tzname = _tf.timezone_at(lng=row.longitude, lat=row.latitude)
    if tzname: # return the name if it is not None
        return (tzname, row.TimeUTC.tz_localize('UTC').tz_convert(tzname))
    return (row.TimeUTC.tz_localize('UTC').tzname(), row.TimeUTC) # else return existing name

chunk[['tzname', 'TimeLocal']] = chunk.apply(lambda row: convert_to_location_tz(row), axis=1, result_type='expand')

In [9]:
chunk

,timestamp,device_aid,device_aid_type,latitude,longitude,horizontal_accuracy,altitude,altitude_accuracy,location_method,ip,user_agent,OS,OS_version,manufacturer,model,carrier,TimeUTC,tzname,TimeLocal
0,1559347200,13f29831-45db-494e-806f-f1df06a69a29,IDFA,59.300000,18.050000,12.5,0.0,NaN,gps,83.250.42.238,Mozilla/5.0 (iPhone; CPU iPhone OS 12_3_1 like...,IOS,12,NaN,iPhone,NaN,2019-06-01 02:00:00,Europe/Stockholm,2019-06-01 04:00:00+02:00
1,1559347200,d2b89545-13c0-452c-afe4-e97fedb1176e,IDFA,58.904400,17.946400,7.7,0.0,NaN,gps,77.53.119.155,Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like M...,IOS,12,NaN,iPhone,NaN,2019-06-01 02:00:00,Europe/Stockholm,2019-06-01 04:00:00+02:00
2,1559347200,417a3389-7a06-4c95-a807-37d9f1b12633,AAID,55.574265,13.040042,12.0,0.0,NaN,cell,5.241.125.16,Mozilla/5.0 (Linux; Android 9; EML-L29 Build/H...,ANDROID,9,NaN,P20,NaN,2019-06-01 02:00:00,Europe/Stockholm,2019-06-01 04:00:00+02:00
3,1559347200,6f685639-1450-4451-b6de-55ea15e93595,AAID,59.460000,17.900000,7.9,0.0,NaN,fused,213.113.87.0,Mozilla/5.0 (Linux; Android 8.0.0; SM-G930F Bu...,ANDROID,8,NaN,Galaxy S7,NaN,2019-06-01 02:00:00,Europe/Stockholm,2019-06-01 04:00:00+02:00
4,1559347200,7ba93f91-eeea-4b2c-8723-d6197f342fff,AAID,59.324700,18.056000,12.8,0.0,NaN,cell,94.191.153.149,Mozilla/5.0 (Linux; Android 7.0; EVA-L19 Build...,ANDROID,7,NaN,P9,NaN,2019-06-01 02:00:00,Europe/Stockholm,2019-06-01 04:00:00+02:00


In [13]:
os.listdir(os.path.join("D:\\mad4abm", "dbs", "raw_data_se_2019", "06", '01'))

['locations-00-part0000.csv.gz',
 'locations-01-part0000.csv.gz',
 'locations-02-part0000.csv.gz',
 'locations-03-part0000.csv.gz',
 'locations-04-part0000.csv.gz',
 'locations-05-part0000.csv.gz',
 'locations-06-part0000.csv.gz',
 'locations-07-part0000.csv.gz',
 'locations-08-part0000.csv.gz',
 'locations-09-part0000.csv.gz',
 'locations-10-part0000.csv.gz',
 'locations-11-part0000.csv.gz',
 'locations-12-part0000.csv.gz',
 'locations-13-part0000.csv.gz',
 'locations-14-part0000.csv.gz',
 'locations-15-part0000.csv.gz',
 'locations-16-part0000.csv.gz',
 'locations-17-part0000.csv.gz',
 'locations-18-part0000.csv.gz',
 'locations-19-part0000.csv.gz',
 'locations-20-part0000.csv.gz',
 'locations-21-part0000.csv.gz',
 'locations-22-part0000.csv.gz',
 'locations-23-part0000.csv.gz']

In [17]:
pd.DataFrame(chunk.device_aid.unique(), columns=['device_aid'])

,device_aid
0,13f29831-45db-494e-806f-f1df06a69a29
1,d2b89545-13c0-452c-afe4-e97fedb1176e
2,417a3389-7a06-4c95-a807-37d9f1b12633
3,6f685639-1450-4451-b6de-55ea15e93595
4,7ba93f91-eeea-4b2c-8723-d6197f342fff


In [21]:
tdf

,datetime,uid,device_aid_type,lat,lng,horizontal_accuracy,altitude,altitude_accuracy,location_method,ip,user_agent,OS,OS_version,manufacturer,model,carrier,TimeUTC,tzname,TimeLocal
0,1970-01-01 00:00:01.559347200,13f29831-45db-494e-806f-f1df06a69a29,IDFA,59.300000,18.050000,12.5,0.0,NaN,gps,83.250.42.238,Mozilla/5.0 (iPhone; CPU iPhone OS 12_3_1 like...,IOS,12,NaN,iPhone,NaN,2019-06-01 02:00:00,Europe/Stockholm,2019-06-01 04:00:00+02:00
1,1970-01-01 00:00:01.559347200,d2b89545-13c0-452c-afe4-e97fedb1176e,IDFA,58.904400,17.946400,7.7,0.0,NaN,gps,77.53.119.155,Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like M...,IOS,12,NaN,iPhone,NaN,2019-06-01 02:00:00,Europe/Stockholm,2019-06-01 04:00:00+02:00
2,1970-01-01 00:00:01.559347200,417a3389-7a06-4c95-a807-37d9f1b12633,AAID,55.574265,13.040042,12.0,0.0,NaN,cell,5.241.125.16,Mozilla/5.0 (Linux; Android 9; EML-L29 Build/H...,ANDROID,9,NaN,P20,NaN,2019-06-01 02:00:00,Europe/Stockholm,2019-06-01 04:00:00+02:00
3,1970-01-01 00:00:01.559347200,6f685639-1450-4451-b6de-55ea15e93595,AAID,59.460000,17.900000,7.9,0.0,NaN,fused,213.113.87.0,Mozilla/5.0 (Linux; Android 8.0.0; SM-G930F Bu...,ANDROID,8,NaN,Galaxy S7,NaN,2019-06-01 02:00:00,Europe/Stockholm,2019-06-01 04:00:00+02:00
4,1970-01-01 00:00:01.559347200,7ba93f91-eeea-4b2c-8723-d6197f342fff,AAID,59.324700,18.056000,12.8,0.0,NaN,cell,94.191.153.149,Mozilla/5.0 (Linux; Android 7.0; EVA-L19 Build...,ANDROID,7,NaN,P9,NaN,2019-06-01 02:00:00,Europe/Stockholm,2019-06-01 04:00:00+02:00


## Check problematic dataset

In [4]:
df = pd.read_csv("dbs/raw_data_se_2019/12/14/locations-06-part0000.csv.gz", sep='\t', compression='gzip')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (0,3,4,5,6,7,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df = df.loc[df['timestamp'] != 'timestamp', :]
df['timestamp'] = df['timestamp'].astype(int)
df.dtypes

timestamp               int32
device_aid             object
device_aid_type        object
latitude               object
longitude              object
horizontal_accuracy    object
altitude               object
altitude_accuracy      object
location_method        object
ip                     object
user_agent             object
OS                     object
OS_version             object
manufacturer           object
model                  object
carrier                object
dtype: object